<a href="https://colab.research.google.com/github/ByeonJaeseong/DeepLearningProject/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -U keras-tuner

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from kerastuner.tuners import RandomSearch
from google.colab import files
import shutil

uploaded = files.upload()



In [ ]:
list = ['s30', 's40', 's50', 's70', 's100', 'c30', 'c40', 'c50', 'c70', 'c100'] #모든 파일에 대해서 for 문 돌려서 실행
count = 1
for i in list :
    #튜너 삭제
    shutil.rmtree('tuner_results')
    # 데이터 불러오기
    lane_data_c = pd.read_csv('lane_data_c.csv', encoding='utf-8')
    lane_data_s = pd.read_csv('lane_data_s.csv', encoding='utf-8')
    data = pd.read_csv('data_'+i+'.csv', encoding='utf-8') #for문을 이용하여 데이터 바꾸기
    # 데이터 결합
    data_combined = pd.concat([lane_data_c, lane_data_s, data], axis=1) #데이터합치기
    data_combined = data_combined.loc[:, ~data_combined.columns.duplicated()]#거리데이터 중복되어있으니까 빼기
    data_combined = data_combined.drop_duplicates(subset='Distance', keep='first')  # 첫 번째 중복 행만 남기기
    # 가중치 계산 함수 정의
    def weighted_mape(y_true, y_pred, weights):
        return np.sum(weights * np.abs((y_true - y_pred) / y_true)) / np.sum(weights) * 100

    # 사용자 정의 Weighted MAPE 손실 함수
    def weighted_mape_loss(weights):
        def loss(y_true, y_pred):
            return tf.reduce_sum(weights * tf.abs((y_true - y_pred) / y_true)) / tf.reduce_sum(weights) * 100
        return loss

    # 입력 변수와 탈선계수 분리
    X_time_series = data_combined[['Distance']]  #Distance를 시계열 데이터로 쓰기 위해서 떼어내기
    X_features = data_combined.drop(['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2', 'Distance'], axis=1) #라벨링데이터 떼어내기
    y = data_combined[['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']] #라벨링하기


    # 데이터 정규화
    scaler = MinMaxScaler() # 스케일링하기
    X_features_scaled = scaler.fit_transform(X_features) #스케일링


    # 학습 데이터와 테스트 데이터 분할
    X_features_train, X_features_test, X_time_series_train, X_time_series_test, y_train, y_test = train_test_split(X_features_scaled, X_time_series, y, test_size=0.2, random_state=42, shuffle=False)

    # 사용자 정의 Weighted MAPE 손실 함수
    def weighted_mape_loss(weights):
        def loss(y_true, y_pred):
            return tf.reduce_sum(weights * tf.abs((y_true - y_pred) / y_true)) / tf.reduce_sum(weights) * 100
        return loss

    # 가중치 계산
    weights_train = np.abs(y_train)  # 훈련 데이터를 기반으로 가중치 계산

    # 입력 정의
    input_time_series = Input(shape=(X_time_series.shape[1], 1), name='input_time_series')
    input_features = Input(shape=(X_features_train.shape[1],), name='input_features')

    # 시계열 데이터 처리를 위한 LSTM 층
    lstm_units = 64
    lstm_output = LSTM(units=lstm_units, activation='tanh', return_sequences=True)(input_time_series)
    lstm_output = LSTM(units=lstm_units, activation='tanh')(lstm_output)

    # 특성 데이터 처리를 위한 밀집층
    features_output = Dense(units=32, activation='relu')(input_features)

    # LSTM 층과 밀집층을 합치기
    concatenated = concatenate([lstm_output, features_output])

    # 예측을 위한 밀집층 추가
    output_layer = Dense(4)(concatenated)  # 4개의 탈선계수를 예측하므로 출력 뉴런 수는 4

    # 모델 구성
    model = Model(inputs=[input_time_series, input_features], outputs=output_layer)

    # 모델 컴파일
    model.compile(optimizer='adam', loss=weighted_mape_loss(weights_train))

    # 하이퍼파라미터 튜닝을 위한 함수 정의
    def build_model(hp):
        lstm_units = hp.Int('lstm_units', min_value=32, max_value=128, step=32)
        dense_units = hp.Int('dense_units', min_value=32, max_value=128, step=32)

        input_time_series = Input(shape=(X_time_series_train.shape[1], 1), name='input_time_series')
        input_features = Input(shape=(X_features_train.shape[1],), name='input_features')

        lstm_output = LSTM(units=lstm_units, activation='relu', return_sequences=True)(input_time_series)
        lstm_output = LSTM(units=lstm_units, activation='relu')(lstm_output)

        features_output = Dense(units=dense_units, activation='relu')(input_features)

        concatenated = concatenate([lstm_output, features_output])

        output_layer = Dense(4)(concatenated)

        tuned_model = Model(inputs=[input_time_series, input_features], outputs=output_layer)
        tuned_model.compile(optimizer='adam', loss='mean_squared_error')

        return tuned_model

    # 하이퍼파라미터 튜닝
    tuner = RandomSearch(build_model, objective='val_loss', max_trials=5, executions_per_trial=1, directory='tuner_results', project_name='model_tuning')
    tuner.search([X_time_series_train, X_features_train], y_train, epochs=50, batch_size=32, validation_data=([X_time_series_test, X_features_test], y_test))
    tuner = RandomSearch(build_model, objective='val_loss', max_trials=5, executions_per_trial=1, directory='tuner_results', project_name='model_tuning')
    tuner.search([X_time_series_train, X_features_train], y_train, epochs=50, batch_size=32, validation_data=([X_time_series_test, X_features_test], y_test))

    # 최적의 모델 선택
    best_model = tuner.get_best_models(num_models=1)[0]

    # 모델 훈련
    best_model.fit([X_time_series_train, X_features_train], y_train, epochs=50, batch_size=32, validation_data=([X_time_series_test, X_features_test], y_test))

    # 다음 1999개의 샘플 예측
    next_samples = 1999
    X_time_series_predict = X_time_series[-next_samples:]
    X_features_predict = X_features[-next_samples:]

    # 모델 예측
    predictions = best_model.predict([X_time_series_predict, X_features_predict])

    answer_sample = pd.read_csv('answer_sample.csv', header=None)
    answer_sample.iloc[1:, count:count+4] = predictions  # 예측 결과 저장
    answer_sample.to_csv('answer_sample.csv', index=False, header=False)  # 결과를 파일에 저장
    count = count+4





Trial 2 Complete [00h 01m 17s]
val_loss: 0.0005834907642565668

Best val_loss So Far: 0.0005781093495897949
Total elapsed time: 00h 02m 25s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
128               |32                |lstm_units
96                |128               |dense_units

Epoch 1/50
300/300 [==============================] - 5s 10ms/step - loss: 2.4604 - val_loss: 0.0105
Epoch 2/50
300/300 [==============================] - 4s 13ms/step - loss: 0.0039 - val_loss: 0.0027
Epoch 3/50
300/300 [==============================] - 3s 8ms/step - loss: 0.0025 - val_loss: 0.0047
Epoch 4/50
300/300 [==============================] - 3s 9ms/step - loss: 0.0016 - val_loss: 0.0044
Epoch 5/50
300/300 [==============================] - 3s 8ms/step - loss: 0.0012 - val_loss: 0.0031
Epoch 6/50
300/300 [==============================] - 3s 9ms/step - loss: 9.0787e-04 - val_loss: 0.0021
Epoch 7/50
300/300 [==============================] - 3s 12ms/step - loss:

In [11]:

answer_sample = pd.read_csv('answer_sample.csv', header=None)
answer_sample.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,Distance,YL_M1_B1_W1_s30,YR_M1_B1_W1_s30,YL_M1_B1_W2_s30,YR_M1_B1_W2_s30,YL_M1_B1_W1_s40,YR_M1_B1_W1_s40,YL_M1_B1_W2_s40,YR_M1_B1_W2_s40,YL_M1_B1_W1_s50,...,YL_M1_B1_W2_c50,YR_M1_B1_W2_c50,YL_M1_B1_W1_c70,YR_M1_B1_W1_c70,YL_M1_B1_W2_c70,YR_M1_B1_W2_c70,YL_M1_B1_W1_c100,YR_M1_B1_W1_c100,YL_M1_B1_W2_c100,YR_M1_B1_W2_c100
1,2500.25,-400.3454284667969,-166.02476501464844,-366.53558349609375,-328.2640075683594,-49.97627639770508,-1.5088118314743042,-45.39625930786133,22.705537796020508,-66.81098937988281,...,-20.428543090820312,-18.646007537841797,616.6464233398438,652.9669189453125,-70.4618911743164,967.9840698242188,-36.86472702026367,-39.732242584228516,-14.5394868850708,16.966903686523438
2,2500.5,-400.72845458984375,-166.37237548828125,-366.3677978515625,-327.69158935546875,-49.45978546142578,-1.2361172437667847,-46.02421951293945,24.00786781311035,-65.3197250366211,...,-20.806840896606445,-17.622617721557617,616.2058715820312,654.3973388671875,-71.30951690673828,968.012451171875,-38.39790725708008,-39.00283432006836,-15.049579620361328,16.96994400024414
3,2500.75,-400.2097473144531,-165.59664916992188,-366.0770568847656,-327.6513366699219,-48.98933410644531,-1.0470608472824097,-45.83039093017578,23.715003967285156,-65.30290985107422,...,-20.614891052246094,-17.41426658630371,615.7760009765625,656.3990478515625,-71.58690643310547,968.3518676757812,-39.57600021362305,-38.34239959716797,-13.354960441589355,15.598198890686035
4,2501.0,-398.69818115234375,-164.1078643798828,-365.8528747558594,-328.3123779296875,-48.77607345581055,-0.44170087575912476,-44.696685791015625,22.893394470214844,-65.8709487915039,...,-20.081937789916992,-18.254657745361328,615.912353515625,657.200439453125,-71.55850219726562,968.072509765625,-39.781585693359375,-37.68818664550781,-12.03775691986084,14.899223327636719
